## Valuing an Option via the Black–Scholes Model

In [6]:
import datetime
import math
import plotly.express as px

from typing import NewType
from typing import Literal

In [7]:
# Define sentinels and consts
Currency = NewType("Currency", float)
OptionType = Literal["call", "put"]
TRADING_DAYS_PER_YEAR = 252

The Standard Normal Distribution CDF (where $\sigma = 1$ and $\mu = 0$ ) is given by 

$$\Phi (x) \coloneqq {\frac {1}{2}}\left(1+\operatorname {erf} \left({\frac {x}{\sqrt {2}}}\right)\right)$$

That is $\Phi(z) = \Pr(Z \leq z)$ where $Z \sim \mathcal{N}(\mu = 0, \sigma^2 = 1)$

In [8]:
# Auxiliary Functions
def std_norm_cdf(x: float) -> float:
    """
    Returns the cumulative distribution function (CDF) of the standard normal distribution.
    """
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

#### Black–Scholes Model
We use the [Black–Scholes model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) to determine the value of a European option given : 

- $r$ - Interest Rate 
- $K$ - Strike Price [Currency] 
- $S$ - Underlying Price [Currency]
- $T$ - Time of option expiration [Years]
- $\sigma$ - Measure of Volatility (square root of the quadratic variation of the stock's log price)  

#### Calculating the Value of Call / Put Options 

Moneyness is the ratio $0 \lt \frac{S}{K} \lt \infty$. For call options if moneyness is $\lt 1$ then we say we are "out of the money" and if its $\gt 1$ we are "in the money".

$d_1$ is a standardized measure of how far the log-moneyness $\ln(S / K)$ may be from zero (i.e. how far it is from breaking even).

$$d_1 = \frac{\ln(S/K) + (r + \tfrac{1}{2}\sigma^2)T}{\sigma \sqrt{T}}$$

$d_2$ shifts $d_1$ down by $\sigma\sqrt{T}$. It represents the standardized moneyness at expiration. 

$$d_2 = d_1 - \sigma \sqrt{T}$$

Using this we express the following for the expected call $C$ and put price $P$ where $\Phi(x)$ is the standard normal distribution. 
$$
C = S \Phi(d_1) - K e^{-rT} \Phi(d_2) \\ \quad \\
P = K e^{-rT} \Phi(-d_2) - S \Phi(-d_1)
$$



In [9]:
def black_sholes(
    interest_rate: float,
    strike_price: Currency,
    underlying_price: Currency,
    time_to_maturity: datetime.timedelta,
    volatility: float,
    option_type: OptionType,
) -> float:
    # Extract the terms
    r = interest_rate
    K = strike_price
    S = underlying_price
    T = time_to_maturity.total_seconds() / (
        TRADING_DAYS_PER_YEAR * 24 * 60 * 60
    )  # cast to years
    sigma = volatility

    # Calculate d1 and d2
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == "call":
        return S * std_norm_cdf(d1) - K * math.exp(-r * T) * std_norm_cdf(d2)

    if option_type == "put":
        return K * math.exp(-r * T) * std_norm_cdf(-d2) - S * std_norm_cdf(-d1)

    raise ValueError(f"Invalid Option Type `{option_type}`")

#### Example


In [11]:
# Example black sholes variables

interest_rate: float = 0.05
strike_price: Currency = Currency(100)
underlying_price: Currency = Currency(100)
time_to_maturity: datetime.timedelta = datetime.timedelta(days=240)
volatility: float = 0.2

black_sholes(
    interest_rate=interest_rate,
    strike_price=strike_price,
    underlying_price=underlying_price,
    time_to_maturity=time_to_maturity,
    volatility=volatility,
    option_type="put",
)

5.492414769106055